In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Генерирование русских имен при помощи RNN

In [ ]:
names = pd.read_csv("./data/name_rus.txt",
            encoding="cp1251", header=None, names=["name"])
names.name = names.name.str.lower().str.strip()
names.head(2)

,name
0,авдокея
1,авдоким


Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

In [ ]:
class Vocab:
    def __init__(self, names):
        xs = names.iloc[:, 0]
        self.max_seq_len = xs.str.len().max()
        tokens = set()
        for name in xs:
            tokens.update(name)    
        self.idx_to_token = dict(enumerate(tokens, 4))
        self.idx_to_token[0] = "<PAD>"
        self.idx_to_token[1] = "<UNK>"
        self.idx_to_token[2] = "<SOS>"
        self.idx_to_token[3] = "<EOS>"
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

In [ ]:
class RusNamesDataset(Dataset):
    def __init__(self, X, vocab: Vocab):
        self.X = X
        self.vocab = vocab

    def vectorize(self, surname):
        surname = surname[:vocab.max_seq_len]
        surname_t = torch.zeros(self.vocab.max_seq_len+2).type(torch.long)
        surname_t += self.vocab.token_to_idx["<PAD>"]
        for i, token in enumerate(surname, 1):
            try:
                surname_t[i] = self.vocab.token_to_idx[token]
            except IndexError as e:
                surname_t[i] = self.vocab.token_to_idx["<UNK>"]
        surname_t[0] = self.vocab.token_to_idx["<SOS>"]
        surname_t[-1] = self.vocab.token_to_idx["<EOS>"]
        return surname_t[:-1], surname_t[1:]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X[idx])

In [ ]:
vocab = Vocab(names)
dataset = RusNamesDataset(names.name.values, vocab)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size] 
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

In [ ]:
class MyModel(nn.Module):
    def __init__(self, vocab_len, embedding_dim, hidden_size, padding_idx,
                 num_layers=1):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(padding_idx=padding_idx,
            num_embeddings=vocab_len, embedding_dim=embedding_dim)
        self.rnn = nn.RNN(num_layers=num_layers, batch_first=True,
            input_size=embedding_dim, hidden_size=hidden_size)
        self.fc = nn.Linear(in_features=hidden_size, out_features=vocab_len)
    def forward(self, x, h=None):
        x = self.embedding(x)
        x, h = self.rnn(x, h)
        x = self.fc(x)
        return x, h

In [ ]:
model = MyModel(
    vocab_len=vocab.vocab_len,
    embedding_dim=32,
    hidden_size=64,
    padding_idx=vocab.token_to_idx["<PAD>"],
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
epoch_step = 10
n_epochs = 101
for epoch in range(n_epochs):
    y_pred = torch.empty(0)
    y_true = torch.empty(0, dtype=torch.long)
    model.train()
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        predictions, h = model(X_batch)
        loss = criterion(predictions.transpose(1, -1), y_batch)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_true = torch.cat((y_true, y_batch))
            y_pred = torch.cat((y_pred, predictions))
    with torch.no_grad():
        train_loss = criterion(y_pred.transpose(1, -1), y_true).item()
    if epoch % epoch_step == 0:
        print(f"#{epoch:3d} --- loss [{train_loss:.4f}]")

# 0 --- loss [1.5506]
#10 --- loss [0.9683]
#20 --- loss [0.8864]
#30 --- loss [0.8391]
#40 --- loss [0.8010]
#50 --- loss [0.7751]
#60 --- loss [0.7506]
#70 --- loss [0.7347]
#80 --- loss [0.7202]
#90 --- loss [0.7059]
#100 --- loss [0.6986]


1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [ ]:
class MyModel(nn.Module):
    def __init__(self, vocab, embedding_dim, hidden_size, num_layers=1):
        super(MyModel, self).__init__()
        self.vocab = vocab
        self.embedding = nn.Embedding(padding_idx=vocab.token_to_idx["<PAD>"],
            num_embeddings=vocab.vocab_len, embedding_dim=embedding_dim)
        self.rnn = nn.RNN(num_layers=num_layers, batch_first=True,
            input_size=embedding_dim, hidden_size=hidden_size)
        self.fc = nn.Linear(
            in_features=hidden_size, out_features=vocab.vocab_len)
    def forward(self, x, h=None):
        x = self.embedding(x)
        x, h = self.rnn(x, h)
        x = self.fc(x)
        return x, h
    
    def random_word(self):
        h = None
        word = []
        token = torch.tensor([[vocab.token_to_idx["<SOS>"]]])
        for _ in range(self.vocab.max_seq_len):
            out, h = self(token, h)
            out_random = out.squeeze().softmax(-1).multinomial(1)
            letter = self.vocab.idx_to_token[out_random.item()]
            if letter == "<EOS>":
                break
            word.append(letter)
            token = out_random.view(1, 1)
        clear = [letter for letter in word if letter not in [
            "<PAD>","<EOS>","<SOS>","<UNK>"]]
        generated = "".join(word).capitalize()
        clear = "".join(clear).capitalize()
        return generated, clear

In [ ]:
model = MyModel(
    vocab=vocab,
    embedding_dim=32,
    hidden_size=64,
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
epoch_step = 10
n_epochs = 101
n_words = 10
for epoch in range(n_epochs):
    y_pred = torch.empty(0)
    y_true = torch.empty(0, dtype=torch.long)
    model.train()
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        predictions, h = model(X_batch)
        loss = criterion(predictions.transpose(1, -1), y_batch)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_true = torch.cat((y_true, y_batch))
            y_pred = torch.cat((y_pred, predictions))
    with torch.no_grad():
        train_loss = criterion(y_pred.transpose(1, -1), y_true).item()
    if epoch % epoch_step == 0:
        print(f"#{epoch:3d} --- loss [{train_loss:.4f}]")
        words=[]
        for _ in range(n_words):
            _, clear = model.random_word()
            words.append(clear)
        print(", ".join(words))

#  0 --- loss [2.2938]
Алтнфб, Алреная, Лиднун, Дфиошиа, Пфрорю, Внйдета, Лчкеваналд, Асдиу, Виафатх, Вмиеаыаа
# 10 --- loss [0.9925]
Фэлюха, Алонтейын, Евося, Линя, Митуша, Ювяй, Инксилья, Трскавр, Веля, Едоша
# 20 --- loss [0.9093]
Корьай, Видора, Гарий, Леяра, Леманьич, Коля, Пегая, Падуня, Селья, Парося
# 30 --- loss [0.8589]
Вермюша, Тома, Валюша, Гетуся, Веля, Темалин, Кольона, Верктушя, Гельюшка, Настадка
# 40 --- loss [0.8169]
Иросеша, Нюхаиктай, Доря, Петюра, Апа, Мулиан, Ваислешка, Леда, Варюня, Гася
# 50 --- loss [0.7931]
Кита, Кирина, Никуля, Вируха, Паврюша, Филюша, Бероша, Варсич, Римуся, Бенюня
# 60 --- loss [0.7668]
Еждя, Диктуша, Нисент, Елюша, Юратюта, Мальб, Нюта, Панюшка, Владиска, Артонина
# 70 --- loss [0.7522]
Маруха, Павра, Алене, Нидилианка, Емилинка, Ситефмианка, Анатима, Лодя, Лаврена, Тюта
# 80 --- loss [0.7333]
Васяна, Ирася, Авдоня, Миняша, Петря, Лизанель, Васяша, Дуся, Маналя, Петря
# 90 --- loss [0.7196]
Антоныч, Коля, Докитка, Дониска, Катанодыч, Макси

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

In [ ]:
import os
os.listdir()

['.config', 'wap2.txt', 'wap4.txt', 'wap1.txt', 'wap3.txt', 'sample_data']

In [ ]:
text = ""
for i in range(1, 5):
    with open(f"./wap{i}.txt", 'r', encoding='cp1251') as f:
        text += f.read()

In [ ]:
class Vocab:
    def __init__(self, text):
        tokens = set(text) 
        self.idx_to_token = dict(enumerate(tokens, 2))
        self.idx_to_token[0] = "<PAD>"
        self.idx_to_token[1] = "<SOS>"
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)

In [ ]:
class BookDataset(Dataset):
    def __init__(self, text, vocab: Vocab, seq_len=128, shift=None):
        self.seq_len = seq_len
        self.shift = shift
        self.X = self.__text_split(text)
        self.vocab = vocab
        
    def __text_split(self, text):
        if self.shift is None: 
            self.shift = self.seq_len // 4
        start = 0
        X = []
        while start < len(text):
            X.append(text[start:start+self.seq_len])
            start += self.shift
        return np.array(X)
    
    def vectorize(self, fragment):
        fragment_t = torch.zeros(self.seq_len+1).long()
        fragment_t += self.vocab.token_to_idx["<PAD>"]
        for i, token in enumerate(fragment, 1):
            fragment_t[i] = self.vocab.token_to_idx[token]
        fragment_t[0] = self.vocab.token_to_idx["<SOS>"]
        return fragment_t[:-1], fragment_t[1:]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.vectorize(self.X[idx])

In [ ]:
vocab = Vocab(text)
dataset = BookDataset(text, vocab)
dataloader = DataLoader(dataset, batch_size=32)

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [ ]:
class MyModel(nn.Module):
    def __init__(self, vocab, embedding_dim, hidden_size, num_layers=1):
        super(MyModel, self).__init__()
        self.vocab = vocab
        self.embedding = nn.Embedding(padding_idx=vocab.token_to_idx["<PAD>"],
            num_embeddings=vocab.vocab_len, embedding_dim=embedding_dim)
        self.rnn = nn.LSTM(num_layers=num_layers, batch_first=True,
            input_size=embedding_dim, hidden_size=hidden_size)
        self.fc = nn.Linear(
            in_features=hidden_size, out_features=vocab.vocab_len)
        
    def forward(self, x, hc=None):
        x = self.embedding(x)
        x, hc = self.rnn(x, hc)
        x = self.fc(x)
        return x, hc
    
    def gen_fragment(self, start_with, length=256):
        hc = None
        word = []
        token = torch.tensor([[vocab.token_to_idx["<SOS>"]]]).to(device)
        for token__ in start_with:
            out, hc = self(token, hc)
            token = torch.tensor(
                [[self.vocab.token_to_idx[token__]]],
                dtype=torch.long
                ).to(device)

        for _ in range(length):
            token = token.to(device)
            out, hc = self(token, hc)
            out_random = out.squeeze().softmax(-1).multinomial(1)
            letter = self.vocab.idx_to_token[out_random.item()]
            word.append(letter)
            token = out_random.view(1, 1)
        clear = [letter for letter in word if letter not in [
            "<PAD>","<EOS>","<SOS>","<UNK>"]]
        generated = "".join(word).capitalize()
        clear = "".join(clear).capitalize()
        return generated, clear

In [ ]:
model = MyModel(
    vocab=vocab,
    embedding_dim=32,
    hidden_size=64,
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
epoch_step = 1
n_epochs = 21
n_words = 10
for epoch in range(n_epochs):
    epoch_loss = 0
    k = 0
    model.train()
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        predictions, h = model(X_batch)
        loss = criterion(predictions.transpose(1, -1), y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        k += 1
    epoch_loss = epoch_loss / k
    if epoch % epoch_step == 0:
        print(f"#{epoch:3d} --- loss [{epoch_loss:.4f}]")
        _, clear = model.gen_fragment(start_with="В начале")
        print("".join(clear))

#  0 --- loss [2.4596]
Щенстистнои как небы быднох, то престаке в к сказиника сьмя, итостарь и выхравни святстя. аплием этом содидичестлек на увиствая схотоврей ни изгралия. кок кобожмоми, дотое потрышать коствение. и стредшим у навожее семы и во таженьюе мосзатичаменных пось то
#  1 --- loss [2.0512]
; друга гриводночсе он седерашно начам проничажанатвает фная, замижении прободния дапера, предошитносо, придильсе его добвула не пошму (не доперживычьим кака... что и еравала изаже поте свете сапраштся. одивлает илик, и принове неми и тозмойках ду тата ник
#  2 --- loss [1.9234]
Коном. во пребедит, что бормию, поратии иболе прожадствия егообстикного сводехста свошля к человек?) наташи, постобитьком, бе косназовина, только чувстви, сердать отстан, с времил его в мака то вляпени присобититься водочно жины сао, на как быти, подама ра
#  3 --- loss [1.8573]
Нный толполосавших постечения извывался пьершения солдатся можение про, но он, - стакантов застредствемандующаяли человени. но человека б

In [ ]:
epoch_step = 1
for epoch in range(21, 41):
    epoch_loss = 0
    k = 0
    model.train()
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        predictions, h = model(X_batch)
        loss = criterion(predictions.transpose(1, -1), y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        k += 1
    epoch_loss = epoch_loss / k
    if epoch % epoch_step == 0:
        print(f"#{epoch:3d} --- loss [{epoch_loss:.4f}]")
        _, clear = model.gen_fragment(start_with="В начале")
        print("".join(clear))

# 21 --- loss [1.6655]
 о собы в события, облама нам сосмычаем жизни, в самое оставлея миромя насказах свободание поряки, которого вадчей монищай, бостасатся начинатие постренноманду, который черезючно законаным данетина: х'ив, что которого напдановники уможно, стоял, спросани; 
# 22 --- loss [1.6630]
М, позамернуют полепанию заей всемимость; нарочит пьера, движение, не было, которых себе, оструется. - франции подумая кучи силье о числом совершенного фидом видел на это было никогда ни в воль по дочеревился не стренно все блюден коренье находям, хорошо) 
# 23 --- loss [1.6607]
. (сноским, он дело отпоражка, необыча, свой, ностветрекой известно держения, говорит, мходействии взялых обрачаем, если и пьера, понявший жится межа только от невнишал (обрадный именять насмейчества этого модшей быть в то, - переп, и событий, законии напо
# 24 --- loss [1.6586]
. встам. воя моя наступии и восточно тор, что они и которая начина и смотреть в собою. а он от танят так же движен наполеонакоками ты чел